<a href="https://colab.research.google.com/github/sunnyna/TIL/blob/master/0717_Document_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 9장. 문서 요약 (Text Summarization)

## Mecab 설치 (필요시)

In [1]:
!sudo apt-get install g++ openjdk-7-jdk # Install Java 1.7+
!sudo apt-get install python-dev; pip install konlpy     # Python 2.x
!sudo apt-get install python3-dev; pip3 install konlpy   # Python 3.x
!sudo apt-get install curl
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Package openjdk-7-jdk is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source

E: Package 'openjdk-7-jdk' has no installation candidate
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-dev is already the newest version (2.7.15~rc1-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
     |████████████████████████████████| 19.4MB 29.7MB/s 
     |████████████████████████████████| 3.8MB 45.6MB/s 
     |████████████████████████████████| 92kB 7.7MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing ins

# 9-1 Luhn Summarizer

http://courses.ischool.berkeley.edu/i256/f06/papers/luhn58.pdf

Hans Peter Luhn

https://en.wikipedia.org/wiki/Hans_Peter_Luhn

 Hans Peter Luhn은 공학과 정보과학에서의 개척 작업으로 "정보 검색의 아버지"로 알려져 있다. 그는 표제어가 문맥에 포함된 채 배열된 색인(KWIC : keyword-in-context) 개발, 정보 선택 제공(SDI), 완전 텍스트 프로세싱, 자동 발췌(요약), 단어 시소로스의 최초 현대식 사용으로 신뢰를 얻었다. 오늘날 파생된 지식 대부분에는 KWIC 색인이 있으며 과학의 모든 분야에 SDI시스템이 있다. 


 Luhn은 1896년 7월 1일 독일 바르멘에서 태어났다. 아버지가 그 당시 유명한 인쇄업자였으므로, 스위스에서 인쇄업을 배웠다. 어렸을 때무터 그는 창조성이 뛰어난 재능을 보였으며, 기술적 문제, 물리학, 통계학에 관심을 보였다. 그러나 1차 세계대전으로 독일군 통신장교로 복역(1915년1917년)하면서 프랑스 터키 루마니아 불가리아 등지로 옮겨 다녀야만 했다. 1차 대전 이후 그는 스위스의 Gallen 교회 Schweizwrische handels Hochschule로 돌아와 기술, 물리학, 회계분야의 수업을 들었다. 그 후, Luhn은 그리스에서 못다한 공부에 전념했으며, 더블부기기계(Duble-Entry Bookeeping Machine : 카드 대장에 대변기입과 차변 기입을 기록할 수 있는 것)를 발명하였다. 그는 또 Hollerith tabulating/recording machine에 정통했고, 천공카드에서 문자 숫자를 나타내는 장치의 사용을 증가시키게 했다. 1920년부터 1930년까지는 10개의 특허권을 획득하여 그의 탁월한 능력을 보여주었다. 그것들중에 루노메터(Lunometer : 직물의 실길이를 계산하는데 쓰이는 장치)는 지금도 사용되고 있다.


 1920년까지 그는 직물 공장에서 일하기 시작했다. 그는 직물 공장의 사업확장을 위해 뉴잉글랜드에서 1924년 미국으로 가게 되었다. 그러나 회사가 곧 파산하였고 Luhn은 직장없이 뉴욕에 남게 되었다. 그는 은행에서 일을 하였고 곧 뉴욕 월스트리트에 소재한 국제어음은행(International Acceptance Bank)에서 재정담당관으로 승진하였다. 


 1933년 Luhn은 자사인 공학회사 H.P. Luhn & Association을 설립하였고 8년간 자문기술자로 일했다. 1941년 Luhn은 IBM에서 수석 연구기술자로 참여하였고 이후에 정보검색연구 관리자로 일했다. Luhn이 IBM에서 새로운 아이디어를 지속적으로 내놓고 문제를 다르게 접근하여 주목을 받는 동안, 다른 기술자들에게 고차원적인 창조를 하도록 자극하면서 그들의 촉매제 역할을 하여 신뢰를 얻었다

In [ ]:
import requests 
from bs4 import BeautifulSoup

def get_news_by_url(url):
  res = requests.get(url)
  bs = BeautifulSoup(res.content, 'html.parser')

  title = bs.select('h3#articleTitle')[0].text #제목
  content = bs.select('#articleBodyContents')[0].get_text().replace('\n', " ") #본문
  content = content.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
  return  content.strip()

doc = get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108')
doc[:50]

'과기정통부, 22일 유영민 장관 등 참석해 기념행사2021년까지 1516억원 투입, 510'

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
 mecab.pos(doc)[:10]

[('과기', 'NNG'),
 ('정통부', 'NNG'),
 (',', 'SC'),
 ('22', 'SN'),
 ('일', 'NNBC'),
 ('유영민', 'NNP'),
 ('장관', 'NNG'),
 ('등', 'NNB'),
 ('참석', 'NNG'),
 ('해', 'XSV+EC')]

In [ ]:
lis=[]
cas=[]
for me, ca in mecab.pos(doc):
  lis.append(me)
  cas.append(ca)
lis[:10]

['과기', '정통부', ',', '22', '일', '유영민', '장관', '등', '참석', '해']

In [ ]:
cas[0][0]

'N'

### 1) 토큰화

In [ ]:
from nltk.tokenize import sent_tokenize
from konlpy.tag import Mecab

mecab = Mecab()
# 문장 분리
def get_sentences(txt): 
  sent_txt = nltk.sent_tokenize(txt)
  return sent_txt

# 토큰화 
def get_words(s):
  tokens = []
  for s in sent_s:
    # token = mecab.pos(s) 
    # tokens.append(token)
    tokens = [token for token, su in mecab.pos(doc) if su[0] in ['N','V']]
    
    
  return tokens

sent_s = get_sentences(doc)
sent_s

tokens = get_words(sent_s)


tokens
# tokens[0][1]

# sentence_tokens = tokens[0]
# sentence_tokens

['과기',
 '정통부',
 '일',
 '유영민',
 '장관',
 '등',
 '참석',
 '기념행사',
 '년',
 '억',
 '원',
 '투입',
 '여종',
 '데이터',
 '구축',
 '민간',
 '클라우드',
 '통한',
 '외부',
 '연계',
 '체계',
 '개방',
 '강화',
 '데일리',
 '이재운',
 '기자',
 '국가',
 '차원',
 '빅',
 '데이터',
 '활용',
 '시대',
 '열린다',
 '새로운',
 '산업',
 '창출',
 '기존',
 '산업',
 '변화',
 '이르',
 '혁신',
 '장',
 '위한',
 '센터',
 '문',
 '연다',
 '개',
 '분야',
 '걸쳐',
 '데이터',
 '경제',
 '발전',
 '위한',
 '정부',
 '청사진',
 '현실',
 '구현',
 '데',
 '앞장선다는',
 '계획',
 '이',
 '일',
 '과학',
 '기술',
 '정보',
 '통신부',
 '서울',
 '중구',
 '대한',
 '상공',
 '회의소',
 '데이터',
 '생태',
 '조성',
 '혁신',
 '성장',
 '기반',
 '마련',
 '위한',
 '빅',
 '데이터',
 '플랫',
 '폼',
 '센터',
 '출범식',
 '행사',
 '개최',
 '유영민',
 '과기',
 '정통부',
 '장관',
 '노웅래',
 '국회',
 '과학',
 '기술',
 '정보',
 '방송',
 '통신',
 '위원회',
 '위원장',
 '등',
 '명',
 '참가',
 '개',
 '분야',
 '개',
 '센터',
 '년',
 '간',
 '억',
 '원',
 '투입',
 '이미지',
 '픽사',
 '베이',
 '빅',
 '데이터',
 '데이터',
 '활용',
 '통해',
 '혁신',
 '장',
 '이루',
 '문재',
 '인',
 '정부',
 '경제',
 '성장',
 '핵심',
 '요소',
 '중',
 '하나',
 '다',
 '문재인',
 '대통령',
 '올',
 '들',
 '데이터',
 '활용',
 '이',
 '따른',
 '정보',
 '보

###문장분리, 토큰화 연습


In [ ]:
sent_text = nltk.sent_tokenize(doc) # this gives us a list of sentences
# now loop over each sentence and tokenize it separately
# for sentence in sent_text:
#     tokenized_text = nltk.word_tokenize(sentence)
#     tagged = nltk.pos_tag(tokenized_text)
#     print(tagged)


In [ ]:
mecab = Mecab()

tokens = []
for s in sentences:
  token = mecab.pos(s)
  
  tokens.append(token)

tokens

# tokens = [token for token in mecab.pos(doc)] #탐색할 토큰 전체
# print(tokens)

[[('과기', 'NNG'),
  ('정통부', 'NNG'),
  (',', 'SC'),
  ('22', 'SN'),
  ('일', 'NNBC'),
  ('유영민', 'NNP'),
  ('장관', 'NNG'),
  ('등', 'NNB'),
  ('참석', 'NNG'),
  ('해', 'XSV+EC'),
  ('기념행사', 'NNG'),
  ('2021', 'SN'),
  ('년', 'NNBC'),
  ('까지', 'JX'),
  ('1516', 'SN'),
  ('억', 'NR'),
  ('원', 'NNBC'),
  ('투입', 'NNG'),
  (',', 'SC'),
  ('5100', 'SN'),
  ('여종', 'NNG'),
  ('데이터', 'NNG'),
  ('구축', 'NNG'),
  ('민간', 'NNG'),
  ('클라우드', 'NNP'),
  ('통한', 'VV+ETM'),
  ('외부', 'NNG'),
  ('연계', 'NNG'),
  ('체계', 'NNG'),
  ('도', 'JX'),
  ('.', 'SF'),
  ('."', 'SY'),
  ('개방', 'NNG'),
  ('성', 'XSN'),
  ('강화', 'NNG'),
  ('"[', 'SY'),
  ('이', 'JKS'),
  ('데일리', 'NNP'),
  ('이재운', 'NNP'),
  ('기자', 'NNG'),
  (']', 'SSC'),
  ('국가', 'NNG'),
  ('차원', 'NNG'),
  ('의', 'JKG'),
  ('빅', 'NNG'),
  ('데이터', 'NNG'),
  ('활용', 'NNG'),
  ('시대', 'NNG'),
  ('가', 'JKS'),
  ('열린다', 'VV+EF'),
  ('.', 'SF')],
 [('새로운', 'VA+ETM'),
  ('산업', 'NNG'),
  ('창출', 'NNG'),
  ('과', 'JC'),
  ('기존', 'NNG'),
  ('산업', 'NNG'),
  ('의', 'JKG'),
  ('변화', 'NNG'

### 2) 중요 단어 결정

In [ ]:
# 단어(토큰)의 가중치 계산 및 범위에 포함되는 토큰 식별 
def get_keywords(word_list , min_ratio=0.001, max_ratio=0.5) :
    

get_keywords(['바나나','사과','바나나','바나나','포도'])

### 전체문서에서 단어의 빈도수 구하기 
단어 리스트를 뽑아서 몇번등장했는지 새기

In [ ]:
# 전체 단어 빈도수 
# single_word= tokens[0][0][0]
sentence_tokens = tokens[0]

# sentence_tokens
token_list = []
for t in tokens:
  token_list.append(t)

print(token_list)
print(len(token_list))
# token_list
# token_list

['과기', '정통부', '일', '유영민', '장관', '등', '참석', '기념행사', '년', '억', '원', '투입', '여종', '데이터', '구축', '민간', '클라우드', '통한', '외부', '연계', '체계', '개방', '강화', '데일리', '이재운', '기자', '국가', '차원', '빅', '데이터', '활용', '시대', '열린다', '새로운', '산업', '창출', '기존', '산업', '변화', '이르', '혁신', '장', '위한', '센터', '문', '연다', '개', '분야', '걸쳐', '데이터', '경제', '발전', '위한', '정부', '청사진', '현실', '구현', '데', '앞장선다는', '계획', '이', '일', '과학', '기술', '정보', '통신부', '서울', '중구', '대한', '상공', '회의소', '데이터', '생태', '조성', '혁신', '성장', '기반', '마련', '위한', '빅', '데이터', '플랫', '폼', '센터', '출범식', '행사', '개최', '유영민', '과기', '정통부', '장관', '노웅래', '국회', '과학', '기술', '정보', '방송', '통신', '위원회', '위원장', '등', '명', '참가', '개', '분야', '개', '센터', '년', '간', '억', '원', '투입', '이미지', '픽사', '베이', '빅', '데이터', '데이터', '활용', '통해', '혁신', '장', '이루', '문재', '인', '정부', '경제', '성장', '핵심', '요소', '중', '하나', '다', '문재인', '대통령', '올', '들', '데이터', '활용', '이', '따른', '정보', '보호', '보안', '대한', '중요', '강조', '했', '맥락', '속', '빅', '데이터', '센터', '공공', '민간', '협업', '활용', '높', '양질', '데이터', '생산', '구축', '플랫', '폼', '이', '수집', '분석'

In [ ]:
from collections import Counter
cnt = Counter(token_list)
cnt.most_common(20)

[('데이터', 31),
 ('센터', 11),
 ('이', 11),
 ('활용', 9),
 ('등', 7),
 ('빅', 7),
 ('개', 7),
 ('플랫', 7),
 ('폼', 7),
 ('구축', 6),
 ('계획', 6),
 ('통해', 6),
 ('과기', 5),
 ('정통부', 5),
 ('년', 5),
 ('원', 5),
 ('혁신', 5),
 ('위한', 5),
 ('분야', 5),
 ('경제', 5)]

### 3) 문장 중요도 계산

In [ ]:
# 문장의 가중치 계산
def get_sentence_weight (sentence , keywords):
    pass

### 4) 문서 요약

In [ ]:
# 문서 요약
def summarize(content ,max_no_of_sentences = 10):
    
    pass

In [ ]:
li = summarize (doc ,  3)
for s in li :
    print(s)



---



# 9-2 Textrank

![대체 텍스트](https://www.researchgate.net/profile/Khushboo_Thakkar3/publication/232645575/figure/fig1/AS:575720050573312@1514273764062/Sample-graph-build-for-sentence-extraction.png)

## 2.1 TextRank 직접 구현하기 (Matrix 활용)

### 1) 자카드 유사도

In [2]:
Text = "딸기 바나나 사과 파인애플. 바나나 사과 딸기 포도. 복숭아 수박. 파인애플 사과 딸기 바나나."

In [7]:
from nltk.tokenize import sent_tokenize
from konlpy.tag import Mecab
import nltk
nltk.download('punkt')

# 문장 분리
def get_sentences(txt):
  return sent_tokenize(txt)

# 토큰화 
def get_words(txt):
  mecab = Mecab()
  return [t[0] for t in mecab.pos(txt)]

get_sentences(Text) 



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['딸기 바나나 사과 파인애플 수박.', '바나나 사과 딸기 포도.', '복숭아 수박.', '파인애플 사과 딸기 바나나.']

In [8]:
from konlpy.tag import Mecab

# 문장간 유사도 측정 (자카드 유사도 사용)

def sentence_similarity(sentence1, sentence2):
  lst1 = get_words(sentence1)
  lst2 = get_words(sentence2)
  
  union = list(set(lst1) | set(lst2))
  intersection = list(set(lst1) & set(lst2))

  return len(intersection) / len(union) 

  

sentence_similarity('나는 치킨을 좋아해','나는 치킨을 싫어해')


0.6666666666666666

### 2) 그래프 생성

In [9]:
 import numpy as np
 def buildMatrix(sentences):
    score = None
    # 문장별로 그래프 edge를 Matrix 형태로 생성
    weighted_edge = None
    
    return score, weighted_edge

Text = "딸기 바나나 사과 파인애플 수박. 바나나 사과 딸기 포도. 복숭아 수박. 파인애플 사과 딸기 바나나."
# buildMatrix(sent_tokenize(Text))    
= sent_tokenize(Text)

['딸기 바나나 사과 파인애플 수박.', '바나나 사과 딸기 포도.', '복숭아 수박.', '파인애플 사과 딸기 바나나.']

In [23]:
import numpy as np
import math

sentences = sent_tokenize(Text)
nodes = len(sentences)

weighted_edge = np.zeros((nodes,nodes))
score = np.ones(len(sentences))


In [32]:


for i in range(nodes):
  for j in range(nodes):
    if i == j:
      continue
    # sen_sim = sentence_similarity(sentences[i],sentences[j])
    # print(f'문장{i+1} -> 문장{j+1} 의 유사도는  {sen_sim}')  # 문장 i+1과 문장 j+1과의 유사도
    weighted_edge[i][j] = sentence_similarity(sentences[i], sentences[j])
    
  for i in range(len(weighted_edge)):
    score[i] = weighted_edge[i].sum()
    weighted_edge[i] /= score[i]


print(weighted_edge)
print(score)

[[0.         0.33802817 0.16901408 0.49295775]
 [0.4137931  0.         0.10344828 0.48275862]
 [0.5        0.25       0.         0.25      ]
 [0.50724638 0.4057971  0.08695652 0.        ]]
[1.         1.         1.         1.64285714]


In [ ]:
# 엣지가중치 계산 (weighted edge)



### 3) 문장 중요도 계산

In [33]:
def scoring(A, P, eps=0.0001, d=0.85, max_iter = 50):
  for iter in range(max_iter):
    newP = (1 -d) + d * (A.T.dot(P))

    if abs((newP - P).sum()) <= eps:
      return newP

    P = newP
  return newP

### 4) 문서 요약

In [34]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [45]:
from nltk.tokenize import sent_tokenize

def summarize(text, n=10):
  text = sent_tokenize(text)
  score_init, weighted_edge = buildMatrix(Text)
  print(score_init, weighted_edge)
  scroe = scoring(weighted_edge, score_init)
  print(score)
  sorted_score = sorted(enumerate(score), key=lambda item: item[1], reverse=True)[:n]
  return [(text[s[0]], s[1]) for s in sorted_score]

In [47]:
summary = summarize(Text, 3)

for sent in summary :
  print(sent)

None None


AttributeError: ignored



---



## 2.2 TextRank 직접 구현하기 (Graph 활용)

In [ ]:
Text = "딸기 바나나 사과 딸기 파인애플. 바나나 사과 딸기. 복숭아 파인애플. 파인애플 사과 딸기 바나나."

### 2) 토큰화

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

In [ ]:
def sentences(text):
  
    return sent_tokenize(text)

In [ ]:
(0.4*0.25*0.25)/(0..4*0.25*0.25)+

### 3) 자카드 유사도

In [ ]:
from konlpy.tag import Mecab

# 문장간 유사도 측정 (자카드 유사도 사용)
def sentence_similarity(sentence1, sentence2):
  pass

sentence_similarity('나는 치킨을 좋아해','나는 치킨을 싫어해')

In [ ]:
def connect(nodes):
    pass

### 3) 그래프 생성

In [48]:
import networkx as nx

def rank(nodes,edges):
    graph = nx.diamond_graph()
    graph.clear()
    graph.add_nodes_from(nodes)
    graph.add_weighted_edge_from(edges)

### 4) 문서 요약

In [ ]:
def summarize(text,num_summaries=3):
    pass


In [ ]:
summary=summarize(Text, 3)

for sent in summary :
  print(sent)

## 2.3 gensim을 활용한 요약

In [ ]:

print(docs[1])

In [ ]:
from gensim.summarization.summarizer import summarize
summarize(docs[1]).split('\n')



---

